In [1]:
import tiktoken

In [ ]:
import pandas as pd

df = pd.read_csv("../DATA/Scraped_Data/medical_tests_interpretation.csv")
df2 = pd.read_csv("../DATA/Scraped_Data/testing_scraped_content.csv")

In [4]:
df.head(2)

,Test Name,Description,Source,URL
0,Acetaminophen Level,What is an acetaminophen level test? This test...,MedlinePlus,https://medlineplus.gov/lab-tests/acetaminophe...
1,Acid-Fast Bacillus (AFB) Tests,What are acid-fast bacillus (AFB) tests? Acid-...,MedlinePlus,https://medlineplus.gov/lab-tests/acid-fast-ba...


In [5]:
def embeddingCreator(text,modelName = "o200k_base"):
    encoding = tiktoken.get_encoding(modelName)
    tokens = encoding.encode(text)
    return tokens

In [6]:
def Scrapper(df,modelName = "o200k_base"):
    df_tokens = pd.DataFrame(columns=["Test Name","tokens","Source","URL"])
    for i in range(len(df)):
        text = df.loc[i,"Description"]
        test_Name = df.loc[i,"Test Name"]
        Url = df.loc[i,"URL"]

        tokens = embeddingCreator(text,modelName)
        df_tokens = pd.concat([df_tokens, pd.DataFrame([{"Test Name":test_Name,"tokens":tokens, "Source":"MedlinePlus.com","URL":Url}])], ignore_index=True)
    return df_tokens

def writeToFile(df_tokens, filename):
    df_tokens.to_csv(filename, index=False)

In [7]:
# Fill NaN values in the Description column with an empty string
df['Description'] = df['Description'].fillna('')
print(df.index)

testing_df = Scrapper(df)

RangeIndex(start=0, stop=300, step=1)


In [10]:
import os

writeToFile(testing_df, "medlinePlus_tokens.csv")

In [19]:
modelName = "o200k_base"
encoding = tiktoken.get_encoding(modelName)

xyz = [5311, 6207, 36493, 1366, 326, 1023, 665, 5025, 2615, 111488, 13, 73234, 21073, 634, 7031, 328, 14140, 4454, 72796, 11, 3101, 13, 8887, 1665, 483, 32629, 4128, 679, 73234, 13, 1843, 481, 679, 32629, 11, 481, 665, 2304, 32629, 45880, 484, 3319, 634, 31907, 2420, 326, 1652, 7332, 481, 591, 4811, 73234, 13, 192212, 484, 4665, 32629, 553, 4358, 4624, 594, 33770, 162537, 17441, 350, 5442, 741, 38638, 8535, 717, 12566, 328, 32629, 9141, 11, 813, 12764, 1309, 316, 2304, 45880, 395, 290, 2867, 328, 634, 2615, 13, 3072, 38638, 665, 3580, 32629, 326, 1652, 481, 4561, 261, 7411, 11, 45649, 2615, 13, 26214, 6953, 32629, 39073, 4700, 10742, 382, 448, 3378, 997, 328, 4137, 3239, 634, 32629, 45880, 553, 13958, 634, 39073, 4700, 4465, 11, 813, 481, 5092, 10509, 13, 10473, 8747, 25, 47081, 291, 18655, 11493, 11, 84778, 11, 47081, 291, 18655, 168100, 1746, 11, 24259, 1228, 11, 32629, 71681, 11, 59607, 4674, 11, 32629, 8237, 3166, 4614, 382, 480, 2061, 395, 30, 1689, 32629, 39073, 4700, 1746, 382, 19294, 2061, 1934, 7163, 40984, 483, 32629, 316, 25, 16975, 14678, 1078, 634, 7448, 6208, 1495, 1775, 634, 32629, 45880, 553, 4113, 15537, 395, 1062, 6629, 306, 634, 32629, 24392, 32629, 39073, 4700, 11493, 382, 1217, 2061, 316, 1746, 66290, 34117, 1261, 1023, 553, 12275, 316, 261, 1647, 1218, 853, 32629, 13, 21926, 2236, 32629, 665, 413, 10292, 316, 261, 10120, 3354, 27943, 1366, 172814, 11, 326, 1819, 23470, 20018, 13, 730, 4454, 7911, 11, 448, 32629, 39073, 4700, 1746, 1340, 413, 2061, 316, 60305]
decodedData = encoding.decode(xyz)
print(decodedData)

 opportunistic infections , and they can become life-threatening. AIDS increases your risk of developing certain cancers, too. Most people with HIV don't have AIDS. If you have HIV, you can take HIV medicines that protect your immune system and help prevent you from getting AIDS. Medicines that treat HIV are called antiretroviral therapy (ART). ART can't get rid of HIV completely, so you'll need to take medicines for the rest of your life. But ART can control HIV and help you live a longer, healthier life. Having regular HIV viral load tests is an important part of making sure your HIV medicines are keeping your viral load low, so you stay healthy. Other names: nucleic acid testing, NAT, nucleic acid amplification test, NAAT, HIV PCR, RNA Test, HIV quantification What is it used for? An HIV viral load test is mainly used after you're diagnosed with HIV to: Guide decisions about your treatment Check how well your HIV medicines are working Watch for any changes in your HIV infection HIV 

In [18]:
len(decodedData)

1304